In [1]:
import mdptoolbox
import numpy as np

In [7]:
states = 10
N = 6
isBadSide = [1,1,1,0,0,0]
p = 1./len(isBadSide)

In [11]:
prob = np.zeros((2, states+2, states+2))

#==== action: continue playing ====

#terminal states
prob[0, states, states] = 1. # lose
prob[0, states+1, states+1] = 1. # win

for i in range(states):
    for side in range(len(isBadSide)):
        if isBadSide[side] == 0:
            cash = side + 1
            if i+cash < states:
                prob[0][i][i+cash] += p
            else:
                prob[0][i][states+1] += p
    prob[0][i][states] += p * sum(isBadSide)

#==== action: get bank ====
prob[1, :, states+1] = 1.

prob[1, states, states+1] = 0
prob[1, states, states] = 1.


#==== check
print(prob[0])

[[ 0.          0.          0.          0.          0.16666667  0.16666667
   0.16666667  0.          0.          0.          0.5         0.        ]
 [ 0.          0.          0.          0.          0.          0.16666667
   0.16666667  0.16666667  0.          0.          0.5         0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.16666667  0.16666667  0.16666667  0.          0.5         0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.16666667  0.16666667  0.16666667  0.5         0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.16666667  0.16666667  0.5         0.16666667]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.16666667  0.5         0.33333333]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.5         0.5      

In [12]:
#rewards = np.zeros((2, states+2, states+2))
#rewards[0, :states, states] = -states
#for i in range(states):
#    rewards[1, i, states+1] = i

rewards = np.zeros((states+2, 2))
rewards[states] = 0
for i in range(states):
    rewards[i, 1] = i

print(rewards)

[[ 0.  0.]
 [ 0.  1.]
 [ 0.  2.]
 [ 0.  3.]
 [ 0.  4.]
 [ 0.  5.]
 [ 0.  6.]
 [ 0.  7.]
 [ 0.  8.]
 [ 0.  9.]
 [ 0.  0.]
 [ 0.  0.]]


In [13]:
vi = mdptoolbox.mdp.ValueIteration(prob, rewards, 1.)
vi.run()

In [14]:
optimal_policy = vi.policy
expected_values = vi.V

print(optimal_policy)
print(expected_values)

(0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0)
(2.5, 3.0, 3.4999999999999996, 4.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 0.0, 0.0)


In [22]:
threshold = 0
for i in range(len(optimal_policy)):
    if optimal_policy[i] == 1:
        threshold = i
        break
threshold

4

In [23]:
# Calc final
def calc_exp(bank=0):
    if bank > threshold:
        return 0
    exp = 0
    for side in range(N):
        if isBadSide[side] == 1:
            exp += -bank * p
        else:
            exp += (side+1) * p + calc_exp(bank+side+1) * p
    return exp

calc_exp()

2.583333333333333